In [ ]:
import pandas as pd
import os
from utils.serialize import mp_serialize_dataframe_for_llm, vp_serialize_dataframe_for_llm
from utils.mappings import mp_column_contexts, twenty_five_mp_questions, vp_column_contexts

mp_df = pd.read_csv("DATA/fake_mp_data.csv")
vp_df = pd.read_csv("DATA/fake_vp_data.csv")

In [ ]:
def has_been_processes(misperid):
    with open("utils/processed.txt") as f:
        scored_ids = [int(id) for id in f.read().split('\n')]
    if misperid in scored_ids:
        return True
    return False

In [ ]:
df = pd.read_pickle("summaries/scores.pkl")
df

,misperid,gpt_4o_summary_base_accuracy,gpt_4o_summary_base_total_potential_accuracy,gpt_4o_summary_base_people_accuracy,gpt_4o_summary_base_people_hallucinations_count,gpt_4o_summary_base_locations_accuracy,gpt_4o_summary_base_locations_hallucinations_count,gpt_4o_summary_template_accuracy,gpt_4o_summary_template_total_potential_accuracy,gpt_4o_summary_template_people_accuracy,gpt_4o_summary_template_people_hallucinations_count,gpt_4o_summary_template_locations_accuracy,gpt_4o_summary_template_locations_hallucinations_count
0,7738,0.074627,0.179104,0.081081,3.0,0.066667,4.0,0.179104,0.268657,0.135135,1.0,0.233333,4.0
1,9465,0.138298,0.244681,0.134615,1.0,0.142857,0.0,0.212766,0.319149,0.134615,1.0,0.309524,4.0
2,1183,0.058824,0.132353,0.050000,3.0,0.071429,2.0,0.294118,0.441176,0.250000,0.0,0.357143,7.0
3,7406,0.052083,0.187500,0.034483,2.0,0.078947,3.0,0.145833,0.333333,0.137931,5.0,0.157895,4.0
4,2983,0.074380,0.165289,0.064935,0.0,0.090909,2.0,0.099174,0.239669,0.077922,3.0,0.136364,4.0
5,714,0.093333,0.173333,0.111111,0.0,0.076923,1.0,0.253333,0.413333,0.277778,0.0,0.230769,5.0
6,6269,0.054054,0.135135,0.050000,6.0,0.058824,6.0,0.202703,0.445946,0.200000,0.0,0.205882,4.0
7,735,0.074074,0.172840,0.090909,2.0,0.054054,4.0,0.160494,0.395062,0.159091,1.0,0.162162,5.0
8,792,0.058252,0.184466,0.056338,1.0,0.062500,6.0,0.106796,0.368932,0.098592,2.0,0.125000,4.0
9,6552,0.109375,0.203125,0.081081,3.0,0.148148,2.0,0.203125,0.375000,0.162162,1.0,0.259259,2.0


In [ ]:
import summarisation_functions
from scoring import score

def simple_summarisation_soring_pipeline(func_name, df_scores, mp_df, vp_df):
    if hasattr(summarisation_functions, func_name):
        func = getattr(summarisation_functions, func_name)
        if not callable(func):
            raise ValueError(f"Function '{func_name}' not working")
    else:
        raise ValueError(f"Function '{func_name}' not found")
    

    misperids = mp_df['misperid'].drop_duplicates().to_list()

    for misperid in misperids: 
        print(f"Processing {misperid} since it has been processed.")
        # if has_been_processes(misperid):
        #     print(f"Not processing{misperid} since it has been processed.")
        #     continue
        if not os.path.exists(f"summaries/{misperid}"):
            os.makedirs(f"summaries/{misperid}")
        
        
        mp_df_misperid = mp_df[mp_df['misperid']==misperid]
        vp_df_misperid = vp_df[vp_df['misper_misperid']==misperid]
        
        mp_serialized = mp_serialize_dataframe_for_llm(mp_df_misperid, mp_column_contexts, twenty_five_mp_questions)
        vp_serialized = vp_serialize_dataframe_for_llm(vp_df_misperid, vp_column_contexts)
        
        # summary = gpt_4o_summary_base(mp_serialized, vp_serialized)
        summary = func(mp_serialized, vp_serialized)
        print(summary)
        raise ValueError("Just test")
        path = f"summaries/{misperid}/"
        with open(path + func_name + '.txt', "w+") as f:
            f.write(summary)
            
        scores, _, _ = score(misperid, "DATA/fake_mp_data.csv", "DATA/fake_vp_data.csv", summary)
        
        # if str(misperid) not in df_scores['misperid']:
        #     df_scores.loc[str(misperid)] = None 
            
        print("A", scores, "A")
        misperid = str(misperid)
        if not (df_scores['misperid'] == misperid).any():
            new_row = {'misperid': misperid}
            df_scores = pd.concat([df_scores, pd.DataFrame([new_row])], ignore_index=True)
        
        for s in scores.keys():
            df_scores.loc[df_scores['misperid'] == misperid, f'{func_name}_{s}'] = scores[s]
    df_scores.to_csv("summaries/scores.csv")
    df_scores.to_pickle("summaries/scores.pkl")
    return df_scores


In [ ]:
# import pandas as pd
# df_scores = pd.read_csv('summaries/scores.csv')
# df_scores

In [ ]:
df = simple_summarisation_soring_pipeline("gpt_4o_summary_template_explanations", df, mp_df, vp_df)

Processing 7738 since it has been processed.
**Missing Person Summary: Kim Wright**

**Basic Information:**
- Name: Kim Wright
- Age: 17 (born June 19, 2007)
- Gender: Female
- Place of Birth: Newry
- Nickname: Dolores

**Vulnerabilities:**
- High-risk due to age (17).
- Hearing impairment noted, which affects well-being and communication.
- Exhibits difficulties interacting safely in unfamiliar situations.
- Previous reports indicate potential mental health concerns linked to her disability.

**Association Network:**
- Familiar individuals: Friends include Lucy Johnson, Sarah Johnson, Robert McCray, and acquaintance Jack Murray.
- Family includes her guardians Ruth Smith and Ryan Wright, who often report her missing.

**Locations:**
- Last known address: 1 St James Square, EH1 3AX.
- Recent traced locations include 5 Buccleuch Place Lane, EH8 9NT, and 1 Tanfield, EH3 5DA.
- Often visits quiet and familiar spots, showing a preference for solitude.

**Patterns of Disappearance:**
- Has 

In [ ]:
df

,misperid,gpt_4o_summary_base_accuracy,gpt_4o_summary_base_total_potential_accuracy,gpt_4o_summary_base_people_accuracy,gpt_4o_summary_base_people_hallucinations_count,gpt_4o_summary_base_locations_accuracy,gpt_4o_summary_base_locations_hallucinations_count,gpt_4o_summary_template_accuracy,gpt_4o_summary_template_total_potential_accuracy,gpt_4o_summary_template_people_accuracy,gpt_4o_summary_template_people_hallucinations_count,gpt_4o_summary_template_locations_accuracy,gpt_4o_summary_template_locations_hallucinations_count,gpt_4o_summary_template_explanations_accuracy,gpt_4o_summary_template_explanations_total_potential_accuracy,gpt_4o_summary_template_explanations_people_accuracy,gpt_4o_summary_template_explanations_people_hallucinations_count,gpt_4o_summary_template_explanations_locations_accuracy,gpt_4o_summary_template_explanations_locations_hallucinations_count
0,7738,0.074627,0.179104,0.081081,3.0,0.066667,4.0,0.179104,0.268657,0.135135,1.0,0.233333,4.0,0.283582,0.417910,0.270270,1.0,0.300000,3.0
1,9465,0.138298,0.244681,0.134615,1.0,0.142857,0.0,0.212766,0.319149,0.134615,1.0,0.309524,4.0,0.170213,0.351064,0.153846,1.0,0.190476,1.0
2,1183,0.058824,0.132353,0.050000,3.0,0.071429,2.0,0.294118,0.441176,0.250000,0.0,0.357143,7.0,0.235294,0.441176,0.250000,0.0,0.214286,3.0
3,7406,0.052083,0.187500,0.034483,2.0,0.078947,3.0,0.145833,0.333333,0.137931,5.0,0.157895,4.0,0.135417,0.291667,0.103448,0.0,0.184211,1.0
4,2983,0.074380,0.165289,0.064935,0.0,0.090909,2.0,0.099174,0.239669,0.077922,3.0,0.136364,4.0,0.115702,0.280992,0.064935,2.0,0.204545,6.0
5,714,0.093333,0.173333,0.111111,0.0,0.076923,1.0,0.253333,0.413333,0.277778,0.0,0.230769,5.0,0.200000,0.466667,0.277778,0.0,0.128205,6.0
6,6269,0.054054,0.135135,0.050000,6.0,0.058824,6.0,0.202703,0.445946,0.200000,0.0,0.205882,4.0,0.189189,0.324324,0.225000,0.0,0.147059,3.0
7,735,0.074074,0.172840,0.090909,2.0,0.054054,4.0,0.160494,0.395062,0.159091,1.0,0.162162,5.0,0.148148,0.345679,0.113636,1.0,0.189189,2.0
8,792,0.058252,0.184466,0.056338,1.0,0.062500,6.0,0.106796,0.368932,0.098592,2.0,0.125000,4.0,0.106796,0.368932,0.084507,2.0,0.156250,1.0
9,6552,0.109375,0.203125,0.081081,3.0,0.148148,2.0,0.203125,0.375000,0.162162,1.0,0.259259,2.0,0.187500,0.296875,0.135135,1.0,0.259259,1.0


In [ ]:
df[['gpt_4o_summary_base_accuracy', 'gpt_4o_summary_template_accuracy', 'gpt_4o_summary_template_explanations_accuracy']]

,gpt_4o_summary_base_accuracy,gpt_4o_summary_template_accuracy,gpt_4o_summary_template_explanations_accuracy
0,0.074627,0.179104,0.283582
1,0.138298,0.212766,0.170213
2,0.058824,0.294118,0.235294
3,0.052083,0.145833,0.135417
4,0.074380,0.099174,0.115702
5,0.093333,0.253333,0.200000
6,0.054054,0.202703,0.189189
7,0.074074,0.160494,0.148148
8,0.058252,0.106796,0.106796
9,0.109375,0.203125,0.187500


In [ ]:
with open("utils/processed.txt") as f:
    scored_ids = [int(id) for id in f.read().split('\n')]
scored_ids

[7738, 9465, 2983, 714, 735, 417, 9585, 5520, 320, 917]